# 🚀 AInote Remote Brain (Colab GPU) - Drive Mode
วิธีใช้: อัพโหลดไฟล์ GGUF ของท่านไปไว้ใน Google Drive แล้วรันสคริปต์นี้

In [ ]:
# 1. Install Dependencies (GPU Enabled)
!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python
!pip install pyngrok uvicorn fastapi sse-starlette pydantic-settings starlette-context

In [ ]:
# 2. Mount Google Drive (เชื่อมต่อไดรฟ์)
from google.colab import drive
drive.mount('/content/drive')

import os

# --- ตั้งค่าชื่อไฟล์ตรงนี้ ---
# ชื่อไฟล์ที่ท่านระบุ: ThaiLLM-8B-Instruct.Q8_0.gguf (ในโฟลเดอร์ Model)
DRIVE_PATH = "/content/drive/MyDrive/Model/ThaiLLM-8B-Instruct.Q8_0.gguf" 

if os.path.exists(DRIVE_PATH):
    print(f"✅ เจอไฟล์โมเดลแล้ว! ({DRIVE_PATH})")
    MODEL_PATH = DRIVE_PATH
else:
    print(f"❌ หาไฟล์ไม่เจอที่: {DRIVE_PATH}")
    print("👉 กรุณาอัพโหลดไฟล์ .gguf ไปไว้ที่ Google Drive ในโฟลเดอร์ Model ก่อนนะครับ")

In [ ]:
# 3. Setup Ngrok & Run Server
from pyngrok import ngrok
import getpass

# --- ใส่ Token แบบปลอดภัย (Interactive) ---
# สมัครฟรีที่ https://dashboard.ngrok.com/get-started/your-authtoken
print("🔑 กรุณาวาง Ngrok Token ของท่านที่นี่ (กด Paste แล้วกด Enter):")
NGROK_TOKEN = getpass.getpass("Insert Ngrok Token: ")

if not NGROK_TOKEN or len(NGROK_TOKEN) < 10:
    print("❌ ท่านยังไม่ได้ใส่ Token ครับ!")
else:
    ngrok.set_auth_token(NGROK_TOKEN)
    
    # Close old tunnels
    ngrok.kill()
    
    # Open Tunnel to port 8000
    public_url = ngrok.connect(8000).public_url
    print(f"\n🚀 COPY ลิ้งก์นี้ไปใส่ใน .env ที่เครื่องครับ: {public_url}/v1\n")
    
    # Run Server (ใช้ไฟล์จาก Drive โดยตรง)
    if 'MODEL_PATH' in locals() and os.path.exists(MODEL_PATH):
        # เพิ่ม --n_ctx 8192 (ขยายสมองเป็น 8192 คำ) เพื่อรับข้อมูลเยอะๆได้
        !python -m llama_cpp.server --model "$MODEL_PATH" --n_gpu_layers -1 --n_ctx 8192 --host 0.0.0.0 --port 8000
    else:
        print("❌ รันไม่ได้เพราะหาไฟล์โมเดลไม่เจอ")